In [1]:
import torch
import neml2
import matplotlib.pyplot as plt
from pyzag import nonlinear, reparametrization, chunktime

In [2]:
torch.set_default_dtype(torch.double)
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [3]:
nrate = 5
nbatch = nrate
max_strain = 0.25
ntime = 100
rates = torch.logspace(-6, 0, nrate, device=device)

In [ ]:
time = torch.zeros((ntime, nrate, 1), device=device)
strain = torch.zeros((ntime, nrate, 6), device=device)
for i, rate in enumerate(rates):
    time[:, i] = torch.linspace(0, max_strain / rate, ntime, device=device)[:, None]
strain[..., 0] = torch.linspace(0, max_strain, ntime, device=device)[:, None]
strain[..., 1] = -0.5 * strain[..., 0]
strain[..., 2] = -0.5 * strain[..., 0]
time = time.reshape((ntime, -1, 1))
strain = strain.reshape((ntime, -1, 6))

RuntimeError: expand(torch.DoubleTensor{[100, 1, 1]}, size=[100, 1]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

In [ ]:
plt.semilogx(time[...,0], strain[..., 0])
plt.xlabel("Time")
plt.ylabel("Strain xx")

In [ ]:
nmodel = neml2.load_model("test_model2.i", "implicit_rate")
nmodel.to(device = device)
stress = torch.zeros((ntime, nbatch, 6), device=device)

In [ ]:
class SolveStrain(torch.nn.Module):
    """Just integrate the model through some strain history

    Args:
        discrete_equations: the pyzag wrapped model
        nchunk (int): number of vectorized time steps
        rtol (float): relative tolerance to use for Newton's method during time integration
        atol (float): absolute tolerance to use for Newton's method during time integration
    """

    def __init__(self, discrete_equations, nchunk=1, rtol=1.0e-6, atol=1.0e-4):
        super().__init__()
        self.discrete_equations = discrete_equations
        self.nchunk = nchunk
        self.rtol = rtol
        self.atol = atol

    def forward(self, time, strain):
        """Integrate through some time/temperature/strain history and return stress
        Args:
            time (torch.tensor): batched times
            strain (torch.tensor): batched strains
        """
        solver = nonlinear.RecursiveNonlinearEquationSolver(
            self.discrete_equations,
            step_generator=nonlinear.StepGenerator(self.nchunk),
            predictor=nonlinear.PreviousStepsPredictor(),
            nonlinear_solver=chunktime.ChunkNewtonRaphson(rtol=self.rtol, atol=self.atol),
        )

        # Setup
        forces = self.discrete_equations.forces_asm.assemble_by_variable(
            {
                "forces/t": time,
                "forces/E": strain,
            }
        ).torch()
        state0 = torch.zeros(
            forces.shape[1:-1] + (self.discrete_equations.nstate,), device=forces.device
        )

        result = nonlinear.solve_adjoint(solver, state0, len(forces), forces)

        return result

In [ ]:
model = SolveStrain(
    neml2.pyzag.NEML2PyzagModel(
        nmodel,
    )
)

In [ ]:
stress = model(time, strain)

In [ ]:
plt.plot(strain[..., 0], stress[...,0].detach())
plt.xlabel("Strain xx")
plt.ylabel("Stress xx")